# TM10 Using Open AI


In [4]:
import pandas as pd
from IPython.display import display
pd.options.display.float_format = '{:.4f}'.format

import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

import re
import zhon.hanzi
from collections import Counter
zh_pat = f"[{zhon.hanzi.characters}|{zhon.hanzi.punctuation}]"

with open("data/stopwords_zh-tw.txt", encoding="utf-8") as fin:
    stopwords = fin.read().split("\n")[1:]

## Step 1. Import and clean data

Notes that we don't need tokenization and stop-word removal here. Just clean up space or punctuation. 

In [5]:
import pandas as pd
df = pd.read_csv('data/sentiment.csv')

def clean_text(text):
    text = re.sub("[\r\t ]+", " ", text)
    text = "".join(re.findall(zh_pat, text))
    return text

df["cleaned"] = df["text"].apply(clean_text)

## Using Open AI

In [27]:
from openai import OpenAI
OPEN_API_KEY = "YOUR_API_KEY"
client = OpenAI(api_key = OPEN_API_KEY)

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
) 

# @retry(wait=wait_random_exponential(min=3, max=10), stop=stop_after_attempt(3))
def ask_gpt(query):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": "以下留言是各類商店討論版中的評論，請協助分析，這些留言的評論對象是"},
            {"role": "system", "content": """
                A-旅館
                B-食品
                C-餐廳
                D-3C用品
                E-其他"""},
            {"role": "system", "content": "只要回答A|B|C|D|E即可，謝謝！你也可以照你的判斷新增新label!"},
            {"role": "user", "content": f"留言內容為「{query}」"},
            {"role": "assistant", "content": "你的答案是："},
            ],
        # n=1,
        # stop=["\n"], 
        temperature=0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return completion
    # return completion.choices[0].message.content.strip()

In [28]:
for i in range(10):
    comment = df["cleaned"][i]
    completion = ask_gpt(comment)
    response = completion.choices[0].message.content.strip()
    print(response, "\t", comment)

"""
D-3C用品 	 店家很給力，快遞也是相當快，第三次光顧啦
D-3C用品 	 這樣的配置用系統還是有點卡。指紋收集器。沒送原裝滑鼠還需要自己買，不太好。
A-旅館 	 不錯，在同等檔次酒店中應該是值得推薦的！
B-食品 	 哎！不會是蒙牛乾的吧嚴懲真凶！
D-3C用品 	 空尤其是三立電視臺女主播做的序尤其無趣像是硬湊那麼多字
D-3C用品 	 明明買了本書，只到了本，也沒有說是什麼原因，以後怎麼信的過？？？？？？？？？？？
E-其他 	 看了一下感覺還可以
D-3C用品 	 散熱還不錯，玩遊戲溫度和硬碟溫度都在以下，速度很好，顯示卡也不錯
D-3C用品 	 外觀好看，白色的自己貼紙也方便，執行起來速度也還不錯屬於主流配置了。一般用用可以的
D-3C用品 	 水超級小用的時候還要先修理一下花灑售後還說是水壓問題說本來標配都是這樣還要自己重新換一個
"""

D-3C用品 	 店家很給力，快遞也是相當快，第三次光顧啦
D-3C用品 	 這樣的配置用系統還是有點卡。指紋收集器。沒送原裝滑鼠還需要自己買，不太好。
A-旅館 	 不錯，在同等檔次酒店中應該是值得推薦的！
B-食品 	 哎！不會是蒙牛乾的吧嚴懲真凶！
D-3C用品 	 空尤其是三立電視臺女主播做的序尤其無趣像是硬湊那麼多字
D-3C用品 	 明明買了本書，只到了本，也沒有說是什麼原因，以後怎麼信的過？？？？？？？？？？？
E-其他 	 看了一下感覺還可以
D-3C用品 	 散熱還不錯，玩遊戲溫度和硬碟溫度都在以下，速度很好，顯示卡也不錯
D-3C用品 	 外觀好看，白色的自己貼紙也方便，執行起來速度也還不錯屬於主流配置了。一般用用可以的
D-3C用品 	 水超級小用的時候還要先修理一下花灑售後還說是水壓問題說本來標配都是這樣還要自己重新換一個
